# Case study
#### Using RISF on real life data from bus sensors

In [2]:
import numpy as np
import pandas as pd
import datetime
import pickle

In [3]:
# Timestamp conversion copied from buses-data-demo.ipynb
VACT_EPOCH = datetime.datetime(year=2011,month=6,day=16,hour=5,minute=23,second=0)
VACT_TIMESTAMP = 1308194580
assert datetime.datetime.fromtimestamp(VACT_TIMESTAMP) == VACT_EPOCH
MILISECOND = 1
SECOND = 1000 * MILISECOND
HOUR = 3600*SECOND
DAY = 24*HOUR
def getVactDate(value):
    return VACT_EPOCH + datetime.timedelta(milliseconds=int(value))

In [4]:
# Loading pandas DataFrame with recorded values from sensors and
# Adding column with transformed timestamp
# Note that this data is ignored by git, so you have to make a copy on your own
# Also the excel file seems to only contain data about the bus number 369
# TODO Remove these links
# Link to data:
# https://halmstaduniversity.box.com/s/rtm3o8dzdt4o0hxr5sredhb1y06qsi8u

def load_data_frames(number):
    file_path = f"../data/bus/data-{number}.pickle"
    df = pickle.load(open(file_path,"rb"))
    df.insert(0, 'tstamp', df.Timestamp.apply(getVactDate))
    globals()[f"DATA_{number}"] = df

In [5]:
# For each number generates DATA_(number) dataframe
buses_numbers = ['369', '370', '371', '372', '375']
for number in buses_numbers:
    load_data_frames(number)

In [6]:
# Example DataFrame
DATA_369

,tstamp,Timestamp,WetTankAirPressure,LongitudAcc,EngineSpeed,Fuel Rate,Engine Load,Boost Pressure,EngineAirInletPressure,AcceleratorPedalPos,VehicleSpeed,BrakePedalPos
0,2012-04-15 08:23:47.763,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
1,2012-04-15 08:23:48.457,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
2,2012-04-15 08:23:49.550,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
3,2012-04-15 08:23:50.715,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
4,2012-04-15 08:23:51.435,2.627645e+10,4.27490,0.0,0.000,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10908295,2014-01-31 19:27:06.824,8.299465e+10,11.65255,-0.2,596.500,4.081143,24.0,0.008618,102.0,0.0,2.331882,0.0
10908296,2014-01-31 19:27:07.994,8.299465e+10,11.58360,-0.4,616.750,3.430526,19.5,0.000000,102.0,0.0,1.796760,11.2
10908297,2014-01-31 19:27:08.672,8.299465e+10,11.51465,0.0,573.000,3.726261,20.5,0.000000,102.0,0.0,0.000000,18.4
10908298,2014-01-31 19:27:09.790,8.299465e+10,11.44570,0.0,617.500,3.726261,20.0,0.000000,102.0,0.0,0.000000,0.0


In [7]:
DATA_369.shape

(10908300, 12)

In [8]:
DATA_369.shape[0] / (4 * 60)

45451.25

In [9]:
# Example entry
DATA_369.iloc[500]

tstamp                    2012-04-15 08:32:07.634000
Timestamp                              26276947634.0
WetTankAirPressure                          10.27355
LongitudAcc                                      0.1
EngineSpeed                                  551.125
Fuel Rate                                   6.801905
Engine Load                                     41.5
Boost Pressure                                   0.0
EngineAirInletPressure                         102.0
AcceleratorPedalPos                             19.2
VehicleSpeed                                1.726452
BrakePedalPos                                    0.0
Name: 500, dtype: object

In [10]:
DATA_369.dtypes

tstamp                    datetime64[ns]
Timestamp                        float64
WetTankAirPressure               float64
LongitudAcc                      float64
EngineSpeed                      float64
Fuel Rate                        float64
Engine Load                      float64
Boost Pressure                   float64
EngineAirInletPressure           float64
AcceleratorPedalPos              float64
VehicleSpeed                     float64
BrakePedalPos                    float64
dtype: object

In [11]:
import altair as alt

start = 26000 + 5000 * 2
chart = alt.Chart(DATA_369.iloc[start:start + 5000].reset_index()).mark_line().encode(
    x='tstamp',
    y='WetTankAirPressure'
).properties(
    width=800
).interactive()

chart

alt.Chart(...)

### Checking hoes in the data

In [12]:
# Function to check real frequency inside the data
def calculate_time_difference(df, datetime_column):
    df['Time_Difference'] = pd.to_datetime(df[datetime_column]).diff().fillna(pd.Timedelta(seconds=0))
    df['Time_Difference'] = df['Time_Difference'].dt.total_seconds()
    return df

In [13]:
# Apply the function to all dataframes 
DATA_369 = calculate_time_difference(DATA_369, 'tstamp')
DATA_370 = calculate_time_difference(DATA_370, 'tstamp')
DATA_371 = calculate_time_difference(DATA_371, 'tstamp')
DATA_372 = calculate_time_difference(DATA_372, 'tstamp')
DATA_375 = calculate_time_difference(DATA_375, 'tstamp')

In [14]:
# Function that would find groups of points that have low frequency (malfunctions of sensor)

def find_outside_interval_groups(df):
    indexes = []
    counts = []
    total_seconds = []

    count = 0
    start_index = None
    current_total_seconds = 0

    for index, value in enumerate(df['Time_Difference']):
        if value < 0.5 or value > 2.5:
        # if value > 2.5:
            if count == 0:
                start_index = index
            count += 1
            current_total_seconds += value
        else:
            if count >= 1:
                indexes.append(start_index)
                counts.append(count)
                total_seconds.append(current_total_seconds)
            count = 0
            current_total_seconds = 0

    data = {'Index': indexes, 'Number of points': counts, 'Total seconds': total_seconds}
    return pd.DataFrame(data)

In [15]:
low_fq_groups_369 = find_outside_interval_groups(DATA_369)
low_fq_groups_370 = find_outside_interval_groups(DATA_370)
low_fq_groups_371 = find_outside_interval_groups(DATA_371)
low_fq_groups_372 = find_outside_interval_groups(DATA_372)
low_fq_groups_375 = find_outside_interval_groups(DATA_375)

In [16]:
def generate_time_bin_table(df):
    bins = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 60, 300, 600, 1800, 7200, 86400, 604800, 2592000, float('inf')]
    labels = ['[0, 0.01] s', '(0.01, 0.1] s', '(0.1, 0.2] s', '(0.2, 0.3] s', '(0.3, 0.4] s', '(0.4, 0.5] s',
              '>(2.5 s, 1 min]', '(1 min, 5 min]', '(5 min, 10 min]', '(10 min, 30 min]', '(30 min, 2 h]',
              '(2 h, 1 day]', '(1 day, 1 week]', '(1 week, 1 month]', '(1 month, infinity)']
    time_bins = pd.cut(df['Total seconds'], bins=bins, labels=labels[:-1], right=False)
    bin_counts = time_bins.value_counts().sort_index()

    table = pd.DataFrame({'Time Bin': bin_counts.index, 'Group Count': bin_counts.values})
    table.reset_index(drop=True, inplace=True)
    return table

In [17]:
time_bin_table_369 = generate_time_bin_table(low_fq_groups_369)
time_bin_table_370 = generate_time_bin_table(low_fq_groups_370)
time_bin_table_371 = generate_time_bin_table(low_fq_groups_371)
time_bin_table_372 = generate_time_bin_table(low_fq_groups_372)
time_bin_table_375 = generate_time_bin_table(low_fq_groups_375)

In [18]:
print(DATA_369.shape[0])
print(DATA_370.shape[0])
print(DATA_371.shape[0])
print(DATA_372.shape[0])
print(DATA_375.shape[0])

10908300
22483955
13213641
16175430
8859806


In [19]:
datasets = [DATA_369, DATA_370, DATA_371, DATA_372, DATA_375]
for dataset in datasets:
    dataset['rounded_tstamp'] = dataset['tstamp'].dt.round('1s')

In [33]:
print(DATA_369['rounded_tstamp'].iloc[0])
print(DATA_369['rounded_tstamp'].iloc[-1])
print('')
print(DATA_370['rounded_tstamp'].iloc[0])
print(DATA_370['rounded_tstamp'].iloc[-1])
print('')
print(DATA_371['rounded_tstamp'].iloc[0])
print(DATA_371['rounded_tstamp'].iloc[-1])
print('')
print(DATA_372['rounded_tstamp'].iloc[0])
print(DATA_372['rounded_tstamp'].iloc[-1])
print('')
print(DATA_375['rounded_tstamp'].iloc[0])
print(DATA_375['rounded_tstamp'].iloc[-1])

print('')
min_time = min(df['rounded_tstamp'].min() for df in datasets)
max_time = max(df['rounded_tstamp'].max() for df in datasets)
print(min_time, max_time)

2012-04-15 08:23:48
2014-01-31 19:27:11

2012-02-01 00:00:01
2014-12-31 00:39:21

2012-01-01 00:00:00
2013-12-31 16:26:07

2012-12-17 22:57:40
2014-12-30 23:45:24

2012-12-17 21:47:54
2014-12-26 19:06:26

2012-01-01 00:00:00 2012-12-17 22:57:49


: 

In [23]:
min_time = min(df['rounded_tstamp'].min() for df in datasets)
max_time = max(df['rounded_tstamp'].max() for df in datasets)

time_range = pd.date_range(start=min_time, end=max_time, freq='1S')

In [17]:
# TODO Remember about changing seed and keeping the seeds consistant
SEED = 23

In [18]:
column_data = DATA_369['WetTankAirPressure'].values[:1000]
X = np.array(column_data)

column_data = DATA_369['WetTankAirPressure'].values[1000:2000]
Y = np.array(column_data)

In [19]:
from risf.forest import RandomIsolationSimilarityForest

RISF = RandomIsolationSimilarityForest(random_state=SEED)

RISF.fit(X)

RISF.predict(Y)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [20]:
# Since the 